In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
import pickle
import os

class DataFrameSpectralProcessor:
    def __init__(self, dataframe=None):
        """
        Initialize processor with optional 2D DataFrame
        Args:
            dataframe: 2D pandas DataFrame representing spectral/image data
        """
        self.original_df = dataframe
        self.current_df = None
        self.data_blocks = {}
        self.processed_spectra = {}
        self.rotation_angle = 0
        
    def load_dataframe(self, dataframe):
        """Load 2D DataFrame"""
        if not isinstance(dataframe, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")
        
        self.original_df = dataframe.copy()
        self.current_df = dataframe.copy()
        
        print(f"DataFrame loaded - Shape: {dataframe.shape}")
        print(f"Data type: {dataframe.dtypes.iloc[0]}")
        print(f"Value range: [{dataframe.min().min():.3f}, {dataframe.max().max():.3f}]")
        return True
    
    def rotate_dataframe(self, angle):
        """
        Rotate DataFrame by specified angle
        Args:
            angle: rotation angle (90, -90, 180)
        """
        if self.current_df is None:
            print("No DataFrame loaded")
            return False
            
        if angle not in [90, -90, 180]:
            print("Angle must be 90, -90, or 180 degrees")
            return False
            
        # Convert DataFrame to numpy array for rotation
        data_array = self.current_df.values
        
        if angle == 90:
            rotated_array = np.rot90(data_array, k=1)  # 90° counterclockwise
        elif angle == -90:
            rotated_array = np.rot90(data_array, k=-1)  # 90° clockwise
        elif angle == 180:
            rotated_array = np.rot90(data_array, k=2)  # 180°
            
        # Create new DataFrame with rotated data
        self.current_df = pd.DataFrame(rotated_array)
        self.rotation_angle = angle
        
        print(f"DataFrame rotated by {angle}° - New shape: {self.current_df.shape}")
        return True
    
    def reset_rotation(self):
        """Reset to original DataFrame"""
        if self.original_df is not None:
            self.current_df = self.original_df.copy()
            self.rotation_angle = 0
            print("DataFrame reset to original orientation")
    
    def slice_data(self, axis='x', start=0, end=None, step_size=1, sum_perpendicular=False):
        """
        Slice DataFrame into blocks
        Args:
            axis: 'x' (columns) or 'y' (rows)
            start, end: slice range
            step_size: size of each block
            sum_perpendicular: whether to sum along perpendicular direction
        """
        if self.current_df is None:
            print("No DataFrame loaded")
            return None
            
        height, width = self.current_df.shape
        
        if axis == 'x':
            max_coord = width
            if end is None: end = width
        else:  # axis == 'y'
            max_coord = height
            if end is None: end = height
                
        # Validate coordinates
        start = max(0, min(start, max_coord - 1))
        end = max(start + 1, min(end, max_coord))
        
        # Calculate number of complete blocks
        total_length = end - start
        num_blocks = total_length // step_size
        
        print(f"Slicing along {axis}-axis from {start} to {end}, step {step_size}")
        print(f"Total blocks: {num_blocks}")
        
        blocks = {}
        
        for i in range(num_blocks):
            if axis == 'x':
                block_start = start + i * step_size
                block_end = start + (i + 1) * step_size
                block_data = self.current_df.iloc[:, block_start:block_end]
                
                if sum_perpendicular:
                    block_data = block_data.sum(axis=0).values  # Sum along rows
            else:  # axis == 'y'
                block_start = start + i * step_size
                block_end = start + (i + 1) * step_size
                block_data = self.current_df.iloc[block_start:block_end, :]
                
                if sum_perpendicular:
                    block_data = block_data.sum(axis=1).values  # Sum along columns
            
            block_name = f"block_{axis}_{i:03d}"
            blocks[block_name] = {
                'data': block_data,
                'axis': axis,
                'start': block_start,
                'end': block_end,
                'summed': sum_perpendicular,
                'shape': block_data.shape if hasattr(block_data, 'shape') else len(block_data)
            }
        
        self.data_blocks.update(blocks)
        print(f"Slicing completed: {len(blocks)} blocks created")
        
        return blocks
    
    def gaussian_func(self, x, amplitude, center, sigma, offset):
        """Gaussian function for fitting"""
        return amplitude * np.exp(-((x - center) ** 2) / (2 * sigma ** 2)) + offset
    
    def lorentzian_func(self, x, amplitude, center, gamma, offset):
        """Lorentzian function for fitting"""
        return amplitude * gamma**2 / ((x - center)**2 + gamma**2) + offset
    
    def find_peaks_and_analyze(self, spectrum, x_axis=None, block_name="spectrum"):
        """
        Comprehensive spectral peak analysis
        Args:
            spectrum: 1D array of spectral data
            x_axis: x-coordinates (optional)
            block_name: identifier for the spectrum
        """
        if x_axis is None:
            x_axis = np.arange(len(spectrum))
        
        # Noise filtering
        smoothed_spectrum = gaussian_filter1d(spectrum, sigma=1.0)
        
        # Peak detection
        peaks, properties = signal.find_peaks(
            smoothed_spectrum, 
            height=np.max(smoothed_spectrum) * 0.1,  # At least 10% of max
            distance=5,  # Minimum distance between peaks
            prominence=np.max(smoothed_spectrum) * 0.05  # Peak prominence
        )
        
        peak_analysis = {}
        
        for i, peak_idx in enumerate(peaks):
            peak_x = x_axis[peak_idx]
            peak_y = smoothed_spectrum[peak_idx]
            
            # Calculate FWHM and 1/e width
            half_max = peak_y / 2
            one_e_height = peak_y / np.e
            
            # Find half-maximum points
            left_idx = peak_idx
            right_idx = peak_idx
            
            # Search left for half-max
            while left_idx > 0 and smoothed_spectrum[left_idx] > half_max:
                left_idx -= 1
            # Search right for half-max
            while right_idx < len(smoothed_spectrum) - 1 and smoothed_spectrum[right_idx] > half_max:
                right_idx += 1
            
            # Interpolate for precise FWHM
            if left_idx < peak_idx:
                left_x = np.interp(half_max, 
                                 [smoothed_spectrum[left_idx], smoothed_spectrum[left_idx + 1]],
                                 [x_axis[left_idx], x_axis[left_idx + 1]])
            else:
                left_x = x_axis[left_idx]
                
            if right_idx > peak_idx:
                right_x = np.interp(half_max,
                                  [smoothed_spectrum[right_idx - 1], smoothed_spectrum[right_idx]],
                                  [x_axis[right_idx - 1], x_axis[right_idx]])
            else:
                right_x = x_axis[right_idx]
            
            fwhm = right_x - left_x
            
            # Calculate 1/e width
            left_1e_idx = peak_idx
            right_1e_idx = peak_idx
            
            while left_1e_idx > 0 and smoothed_spectrum[left_1e_idx] > one_e_height:
                left_1e_idx -= 1
            while right_1e_idx < len(smoothed_spectrum) - 1 and smoothed_spectrum[right_1e_idx] > one_e_height:
                right_1e_idx += 1
                
            # Interpolate 1/e points
            if left_1e_idx < peak_idx:
                left_1e_x = np.interp(one_e_height,
                                    [smoothed_spectrum[left_1e_idx], smoothed_spectrum[left_1e_idx + 1]],
                                    [x_axis[left_1e_idx], x_axis[left_1e_idx + 1]])
            else:
                left_1e_x = x_axis[left_1e_idx]
                
            if right_1e_idx > peak_idx:
                right_1e_x = np.interp(one_e_height,
                                     [smoothed_spectrum[right_1e_idx - 1], smoothed_spectrum[right_1e_idx]],
                                     [x_axis[right_1e_idx - 1], x_axis[right_1e_idx]])
            else:
                right_1e_x = x_axis[right_1e_idx]
                
            width_1e = right_1e_x - left_1e_x
            
            # Curve fitting (Gaussian and Lorentzian)
            try:
                fit_range = max(10, int(fwhm * 2)) if fwhm > 0 else 10
                fit_start = max(0, peak_idx - fit_range)
                fit_end = min(len(spectrum), peak_idx + fit_range)
                
                x_fit = x_axis[fit_start:fit_end]
                y_fit = smoothed_spectrum[fit_start:fit_end]
                
                # Gaussian fit
                p0_gauss = [peak_y, peak_x, fwhm/2.355, np.min(y_fit)]
                popt_gauss, _ = curve_fit(self.gaussian_func, x_fit, y_fit, p0=p0_gauss, maxfev=1000)
                fitted_fwhm_gauss = 2.355 * abs(popt_gauss[2])
                
                # Lorentzian fit
                p0_lor = [peak_y, peak_x, fwhm/2, np.min(y_fit)]
                popt_lor, _ = curve_fit(self.lorentzian_func, x_fit, y_fit, p0=p0_lor, maxfev=1000)
                fitted_fwhm_lor = 2 * abs(popt_lor[2])
                
            except:
                popt_gauss = None
                popt_lor = None
                fitted_fwhm_gauss = None
                fitted_fwhm_lor = None
            
            # Peak area calculation
            peak_area = np.trapz(smoothed_spectrum[left_idx:right_idx+1], 
                               x_axis[left_idx:right_idx+1])
            
            # Signal-to-noise ratio
            noise_level = np.std(smoothed_spectrum[:min(50, len(smoothed_spectrum))])
            snr = peak_y / noise_level if noise_level > 0 else float('inf')
            
            peak_info = {
                'peak_index': peak_idx,
                'peak_position': peak_x,
                'peak_intensity': peak_y,
                'fwhm_measured': fwhm,
                'width_1e': width_1e,
                'fwhm_gaussian_fit': fitted_fwhm_gauss,
                'fwhm_lorentzian_fit': fitted_fwhm_lor,
                'gaussian_params': popt_gauss,
                'lorentzian_params': popt_lor,
                'peak_area': peak_area,
                'snr': snr,
                'prominence': properties['prominences'][i] if i < len(properties['prominences']) else None
            }
            
            peak_analysis[f'peak_{i+1}'] = peak_info
        
        # Store analysis results
        analysis_result = {
            'original_spectrum': spectrum,
            'smoothed_spectrum': smoothed_spectrum,
            'x_axis': x_axis,
            'peaks': peak_analysis,
            'num_peaks': len(peaks)
        }
        
        self.processed_spectra[block_name] = analysis_result
        return analysis_result
    
    def visualize_spectrum_analysis(self, block_name):
        """Visualize spectral analysis results"""
        if block_name not in self.processed_spectra:
            print(f"No analysis results found for {block_name}")
            return
            
        result = self.processed_spectra[block_name]
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Top plot: original and smoothed spectra
        ax1.plot(result['x_axis'], result['original_spectrum'], 'lightblue', alpha=0.7, label='Original')
        ax1.plot(result['x_axis'], result['smoothed_spectrum'], 'blue', linewidth=2, label='Smoothed')
        
        # Mark peaks
        for peak_name, peak_info in result['peaks'].items():
            ax1.plot(peak_info['peak_position'], peak_info['peak_intensity'], 'ro', markersize=8)
            ax1.annotate(f"{peak_name}\nPos: {peak_info['peak_position']:.2f}", 
                        (peak_info['peak_position'], peak_info['peak_intensity']),
                        xytext=(10, 10), textcoords='offset points', fontsize=8)
        
        ax1.set_title(f'Spectral Analysis - {block_name}')
        ax1.set_xlabel('Position/Wavelength')
        ax1.set_ylabel('Intensity')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Bottom plot: peak width comparison
        if result['peaks']:
            peak_names = list(result['peaks'].keys())
            fwhms = [result['peaks'][name]['fwhm_measured'] for name in peak_names]
            width_1es = [result['peaks'][name]['width_1e'] for name in peak_names]
            
            x_pos = np.arange(len(peak_names))
            width = 0.35
            
            ax2.bar(x_pos - width/2, fwhms, width, label='FWHM', alpha=0.8)
            ax2.bar(x_pos + width/2, width_1es, width, label='1/e Width', alpha=0.8)
            
            ax2.set_xlabel('Peaks')
            ax2.set_ylabel('Width')
            ax2.set_title('Peak Width Comparison')
            ax2.set_xticks(x_pos)
            ax2.set_xticklabels(peak_names, rotation=45)
            ax2.legend()
            ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed results
        print(f"\n=== {block_name} Analysis Results ===")
        for peak_name, peak_info in result['peaks'].items():
            print(f"\n{peak_name}:")
            print(f"  Position: {peak_info['peak_position']:.3f}")
            print(f"  Intensity: {peak_info['peak_intensity']:.3f}")
            print(f"  FWHM (measured): {peak_info['fwhm_measured']:.3f}")
            print(f"  1/e width: {peak_info['width_1e']:.3f}")
            if peak_info['fwhm_gaussian_fit']:
                print(f"  FWHM (Gaussian fit): {peak_info['fwhm_gaussian_fit']:.3f}")
            if peak_info['fwhm_lorentzian_fit']:
                print(f"  FWHM (Lorentzian fit): {peak_info['fwhm_lorentzian_fit']:.3f}")
            print(f"  SNR: {peak_info['snr']:.2f}")
    
    def visualize_dataframe(self):
        """Visualize current DataFrame"""
        if self.current_df is None:
            print("No DataFrame loaded")
            return
            
        fig, ax = plt.subplots(figsize=(10, 8))
        img = ax.imshow(self.current_df.values, cmap='gray', aspect='auto')
        plt.colorbar(img, ax=ax, label='Intensity')
        
        title = f'DataFrame Visualization'
        if self.rotation_angle != 0:
            title += f' (Rotated {self.rotation_angle}°)'
        
        ax.set_title(title)
        ax.set_xlabel('Columns')
        ax.set_ylabel('Rows')
        plt.show()
    
    def save_analysis_results(self, filename="spectral_analysis_results.pkl"):
        """Save analysis results to file"""
        save_data = {
            'data_blocks': self.data_blocks,
            'processed_spectra': self.processed_spectra,
            'rotation_angle': self.rotation_angle,
            'dataframe_shape': self.current_df.shape if self.current_df is not None else None
        }
        
        with open(filename, 'wb') as f:
            pickle.dump(save_data, f)
        
        print(f"Analysis results saved to: {filename}")
    
    def load_analysis_results(self, filename):
        """Load previously saved analysis results"""
        try:
            with open(filename, 'rb') as f:
                save_data = pickle.load(f)
            
            self.data_blocks = save_data.get('data_blocks', {})
            self.processed_spectra = save_data.get('processed_spectra', {})
            self.rotation_angle = save_data.get('rotation_angle', 0)
            
            print(f"Analysis results loaded from: {filename}")
            print(f"Loaded {len(self.data_blocks)} data blocks and {len(self.processed_spectra)} spectral analyses")
            
        except Exception as e:
            print(f"Error loading file: {e}")
    
    def list_data_blocks(self):
        """List all available data blocks"""
        if not self.data_blocks:
            print("No data blocks available")
            return
            
        print("Available data blocks:")
        for name, block in self.data_blocks.items():
            print(f"  - {name}: shape {block['shape']}, axis {block['axis']}, "
                  f"range [{block['start']}:{block['end']}], summed: {block['summed']}")
    
    def list_processed_spectra(self):
        """List all processed spectral analyses"""
        if not self.processed_spectra:
            print("No processed spectra available")
            return
            
        print("Available spectral analyses:")
        for name, result in self.processed_spectra.items():
            print(f"  - {name}: {result['num_peaks']} peaks detected")

# Main program
if __name__ == "__main__":
    processor = DataFrameSpectralProcessor()
    # Note: Load your DataFrame here with processor.load_dataframe(your_df)
    print("DataFrame Spectral Processor initialized")
    print("Use processor.load_dataframe(your_df) to start")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
import pickle
import os

class DataFrameSpectralProcessor:
    def __init__(self, dataframe=None):
        """
        使用可选的 2D DataFrame 初始化处理器
        参数:
            dataframe: 表示光谱/图像数据的 2D pandas DataFrame
        """
        self.original_df = dataframe
        self.current_df = None
        self.data_blocks = {}
        self.processed_spectra = {}
        self.rotation_angle = 0
        
    def load_dataframe(self, dataframe):
        """加载 2D DataFrame"""
        if not isinstance(dataframe, pd.DataFrame):
            raise ValueError("输入必须是 pandas DataFrame 类型")
        
        self.original_df = dataframe.copy()
        self.current_df = dataframe.copy()
        
        print(f"DataFrame 已加载 - 形状: {dataframe.shape}")
        print(f"数据类型: {dataframe.dtypes.iloc[0]}")
        print(f"数值范围: [{dataframe.min().min():.3f}, {dataframe.max().max():.3f}]")
        return True
    
    def rotate_dataframe(self, angle):
        """
        将 DataFrame 旋转指定角度
        参数:
            angle: 旋转角度（90、-90 或 180）
        """
        if self.current_df is None:
            print("尚未加载 DataFrame")
            return False
            
        if angle not in [90, -90, 180]:
            print("角度必须是 90、-90 或 180 度")
            return False
            
        # 将 DataFrame 转换为 numpy 数组以进行旋转
        data_array = self.current_df.values
        
        if angle == 90:
            rotated_array = np.rot90(data_array, k=1)  # 逆时针旋转 90°
        elif angle == -90:
            rotated_array = np.rot90(data_array, k=-1)  # 顺时针旋转 90°
        elif angle == 180:
            rotated_array = np.rot90(data_array, k=2)  # 旋转 180°
            
        # 用旋转后的数据创建新的 DataFrame
        self.current_df = pd.DataFrame(rotated_array)
        self.rotation_angle = angle
        
        print(f"DataFrame 已旋转 {angle}° - 新形状: {self.current_df.shape}")
        return True
    
    def reset_rotation(self):
        """重置为原始 DataFrame"""
        if self.original_df is not None:
            self.current_df = self.original_df.copy()
            self.rotation_angle = 0
            print("DataFrame 已重置为原始方向")
    
    def slice_data(self, axis='x', start=0, end=None, step_size=1, sum_perpendicular=False):
        """
        将 DataFrame 按块切片
        参数:
            axis: 'x'（按列切）或 'y'（按行切）
            start, end: 切片范围
            step_size: 每个块的大小
            sum_perpendicular: 是否沿垂直方向求和
        """
        if self.current_df is None:
            print("尚未加载 DataFrame")
            return None
            
        height, width = self.current_df.shape
        
        if axis == 'x':
            max_coord = width
            if end is None: end = width
        else:  # axis == 'y'
            max_coord = height
            if end is None: end = height
                
        # 校验坐标
        start = max(0, min(start, max_coord - 1))
        end = max(start + 1, min(end, max_coord))
        
        # 计算完整块的数量
        total_length = end - start
        num_blocks = total_length // step_size
        
        print(f"沿 {axis}-轴切片，从 {start} 到 {end}，步长为 {step_size}")
        print(f"总块数: {num_blocks}")
        
        blocks = {}
        
        for i in range(num_blocks):
            if axis == 'x':
                block_start = start + i * step_size
                block_end = start + (i + 1) * step_size
                block_data = self.current_df.iloc[:, block_start:block_end]
                
                if sum_perpendicular:
                    block_data = block_data.sum(axis=0).values  # 沿行方向求和
            else:  # axis == 'y'
                block_start = start + i * step_size
                block_end = start + (i + 1) * step_size
                block_data = self.current_df.iloc[block_start:block_end, :]
                
                if sum_perpendicular:
                    block_data = block_data.sum(axis=1).values  # 沿列方向求和
            
            block_name = f"block_{axis}_{i:03d}"
            blocks[block_name] = {
                'data': block_data,
                'axis': axis,
                'start': block_start,
                'end': block_end,
                'summed': sum_perpendicular,
                'shape': block_data.shape if hasattr(block_data, 'shape') else len(block_data)
            }
        
        self.data_blocks.update(blocks)
        print(f"切片完成：共创建 {len(blocks)} 个数据块")
        
        return blocks
    
    def gaussian_func(self, x, amplitude, center, sigma, offset):
        """用于拟合的高斯函数"""
        return amplitude * np.exp(-((x - center) ** 2) / (2 * sigma ** 2)) + offset
    
    def lorentzian_func(self, x, amplitude, center, gamma, offset):
        """用于拟合的洛伦兹函数"""
        return amplitude * gamma**2 / ((x - center)**2 + gamma**2) + offset
    
    def find_peaks_and_analyze(self, spectrum, x_axis=None, block_name="spectrum"):
        """
        全面的光谱峰分析
        参数:
            spectrum: 一维光谱数据
            x_axis: x 坐标（可选）
            block_name: 光谱标识符
        """
        if x_axis is None:
            x_axis = np.arange(len(spectrum))
        
        # 噪声滤波
        smoothed_spectrum = gaussian_filter1d(spectrum, sigma=1.0)
        
        # 峰值检测
        peaks, properties = signal.find_peaks(
            smoothed_spectrum, 
            height=np.max(smoothed_spectrum) * 0.1,  # 至少为最大值的10%
            distance=5,  # 峰值之间的最小距离
            prominence=np.max(smoothed_spectrum) * 0.05  # 峰值显著性
        )
        
        peak_analysis = {}
        
        for i, peak_idx in enumerate(peaks):
            peak_x = x_axis[peak_idx]
            peak_y = smoothed_spectrum[peak_idx]
            
            # 计算 FWHM 和 1/e 宽度
            half_max = peak_y / 2
            one_e_height = peak_y / np.e
            
            # 查找半高宽点
            left_idx = peak_idx
            right_idx = peak_idx
            
            # 向左查找半高点
            while left_idx > 0 and smoothed_spectrum[left_idx] > half_max:
                left_idx -= 1
            # 向右查找半高点
            while right_idx < len(smoothed_spectrum) - 1 and smoothed_spectrum[right_idx] > half_max:
                right_idx += 1
            
            # 插值计算 FWHM 边界
            if left_idx < peak_idx:
                left_x = np.interp(half_max, 
                                 [smoothed_spectrum[left_idx], smoothed_spectrum[left_idx + 1]],
                                 [x_axis[left_idx], x_axis[left_idx + 1]])
            else:
                left_x = x_axis[left_idx]
                
            if right_idx > peak_idx:
                right_x = np.interp(half_max,
                                  [smoothed_spectrum[right_idx - 1], smoothed_spectrum[right_idx]],
                                  [x_axis[right_idx - 1], x_axis[right_idx]])
            else:
                right_x = x_axis[right_idx]
            
            fwhm = right_x - left_x
            
            # 计算 1/e 宽度
            left_1e_idx = peak_idx
            right_1e_idx = peak_idx
            
            while left_1e_idx > 0 and smoothed_spectrum[left_1e_idx] > one_e_height:
                left_1e_idx -= 1
            while right_1e_idx < len(smoothed_spectrum) - 1 and smoothed_spectrum[right_1e_idx] > one_e_height:
                right_1e_idx += 1
                
            # 插值计算 1/e 边界
            if left_1e_idx < peak_idx:
                left_1e_x = np.interp(one_e_height,
                                    [smoothed_spectrum[left_1e_idx], smoothed_spectrum[left_1e_idx + 1]],
                                    [x_axis[left_1e_idx], x_axis[left_1e_idx + 1]])
            else:
                left_1e_x = x_axis[left_1e_idx]
                
            if right_1e_idx > peak_idx:
                right_1e_x = np.interp(one_e_height,
                                     [smoothed_spectrum[right_1e_idx - 1], smoothed_spectrum[right_1e_idx]],
                                     [x_axis[right_1e_idx - 1], x_axis[right_1e_idx]])
            else:
                right_1e_x = x_axis[right_1e_idx]
                
            width_1e = right_1e_x - left_1e_x
            
            # 曲线拟合（高斯和洛伦兹）
            try:
                fit_range = max(10, int(fwhm * 2)) if fwhm > 0 else 10
                fit_start = max(0, peak_idx - fit_range)
                fit_end = min(len(spectrum), peak_idx + fit_range)
                
                x_fit = x_axis[fit_start:fit_end]
                y_fit = smoothed_spectrum[fit_start:fit_end]
                
                # 高斯拟合
                p0_gauss = [peak_y, peak_x, fwhm/2.355, np.min(y_fit)]
                popt_gauss, _ = curve_fit(self.gaussian_func, x_fit, y_fit, p0=p0_gauss, maxfev=1000)
                fitted_fwhm_gauss = 2.355 * abs(popt_gauss[2])
                
                # 洛伦兹拟合
                p0_lor = [peak_y, peak_x, fwhm/2, np.min(y_fit)]
                popt_lor, _ = curve_fit(self.lorentzian_func, x_fit, y_fit, p0=p0_lor, maxfev=1000)
                fitted_fwhm_lor = 2 * abs(popt_lor[2])
                
            except:
                popt_gauss = None
                popt_lor = None
                fitted_fwhm_gauss = None
                fitted_fwhm_lor = None
            
            # 计算峰面积
            peak_area = np.trapz(smoothed_spectrum[left_idx:right_idx+1], 
                               x_axis[left_idx:right_idx+1])
            
            # 信噪比
            noise_level = np.std(smoothed_spectrum[:min(50, len(smoothed_spectrum))])
            snr = peak_y / noise_level if noise_level > 0 else float('inf')
            
            peak_info = {
                'peak_index': peak_idx,
                'peak_position': peak_x,
                'peak_intensity': peak_y,
                'fwhm_measured': fwhm,
                'width_1e': width_1e,
                'fwhm_gaussian_fit': fitted_fwhm_gauss,
                'fwhm_lorentzian_fit': fitted_fwhm_lor,
                'gaussian_params': popt_gauss,
                'lorentzian_params': popt_lor,
                'peak_area': peak_area,
                'snr': snr,
                'prominence': properties['prominences'][i] if i < len(properties['prominences']) else None
            }
            
            peak_analysis[f'peak_{i+1}'] = peak_info
        
        # 存储分析结果
        analysis_result = {
            'original_spectrum': spectrum,
            'smoothed_spectrum': smoothed_spectrum,
            'x_axis': x_axis,
            'peaks': peak_analysis,
            'num_peaks': len(peaks)
        }
        
        self.processed_spectra[block_name] = analysis_result
        return analysis_result
